##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License { display-mode: "form" }
#
# Copyright (c) 2018 Dr. Kashif Rasul
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Estimators and multi-GPUs with tf.keras, tf.data, and eager execution

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/estimators/keras-estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/models/blob/master/samples/core/tutorials/estimators/keras-estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/models/blob/master/samples/core/tutorials/estimators/keras-estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

The [Estimators](https://www.tensorflow.org/guide/estimators) API is used for training models in a distributed enviornment such as on nodes with multiple GPUs or on many nodes with GPUs etc. This is particularly useful when training on huge datasets. But we will however present the API for the tiny Fashion-MNIST dataset as usual in the interest of time.

Essentially what we want to remember is that a `tf.keras.Model` can be trained with `tf.estimator` API by converting it to `tf.estimator.Estimator` object via the `tf.keras.estimator.model_to_estimator` method. Once converted we can apply the machinery of the `Estimator` to train on different hardware configurations.

In [0]:
!pip install -U tf-nightly-gpu

In [0]:
import tensorflow as tf

import numpy as np

tf.enable_eager_execution()

## Import the dataset

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [0]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

train_images = np.asarray(train_images, dtype=np.float32) / 255

# Convert the train images and add channels
train_images = train_images.reshape((TRAINING_SIZE, 28, 28, 1))

test_images = np.asarray(test_images, dtype=np.float32) / 255
# Convert the train images and add channels
test_images = test_images.reshape((TEST_SIZE, 28, 28, 1))

In [0]:
# How many categories we are predicting from (0-9)
LABEL_DIMENSIONS = 10

train_labels  = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels, LABEL_DIMENSIONS)

# Cast the labels to floats, needed later
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

## Build a Keras model

In [0]:
inputs = tf.keras.Input(shape=(28,28,1))  # Returns a placeholder tensor
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu)(inputs)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation=tf.nn.relu)(x)
predictions = tf.keras.layers.Dense(LABEL_DIMENSIONS, activation=tf.nn.softmax)(x)

In [0]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)

In [0]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

## Create an Estimator

Now to create an Estimator from the compiled Keras model we call the `model_to_estimator` method. Note that the initial model state of the keras model is preserved in the created Estimator.

So what's so good about Estimators? Well to start off with:

* you can run Estimator-based models on a local host or an a distributed multi-GPU environment without changing your model
* Estimators simplify sharing implementations between model developers
* Estimators build the graph for you


So how do we go about training our simple `tf.keras` model to use multi-GPUs? Well we can use the `tf.contrib.MirroredStrategy` paradigm which does in-graph replication with synchronous training. To use this we first create an estimator from the `tf.keras` model and give it the `MirroredStrategy` configuration via the `RunConfig`:

In [0]:
strategy = tf.contrib.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

estimator = tf.keras.estimator.model_to_estimator(model, config=config)

## Create an Estimator input function

To pipe data into Estimators we need to define a data importing function like shown below which returns batches of `(images, labels)` as shown below:

In [0]:
def input_fn(images, labels, repeat, batch_size):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))

    # Shuffle, repeat, and batch the examples.
    SHUFFLE_SIZE = 10000
    dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(repeat).batch(batch_size)

    # Return the dataset.
    return dataset

## Train the Estimator

Now the good part! We can call the `train` function on our estimator giving it `input_fn` we defined as well as the number of steps of the optimizer we wish to run:

In [0]:
BATCH_SIZE=128
EPOCHS=5
STEPS = 1000

estimator.train(input_fn=lambda:input_fn(train_images,
                                         train_labels,
                                         repeat=EPOCHS,
                                         batch_size=BATCH_SIZE), 
                steps=STEPS)

## Evaluate the Estimator

In order to check the performance of our model we can then run the `evaluate` method on our Estimator:

In [0]:
estimator.evaluate(input_fn=lambda:input_fn(test_images, test_labels, repeat=1, batch_size=BATCH_SIZE), steps=1)